# importing libraries

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cd drive/My\ Drive/Projet\ Altegrad

/content/drive/My Drive/Projet Altegrad


In [0]:
import warnings
warnings.filterwarnings("ignore")

In [0]:
import os
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import codecs

from os import path
from functions_text import *
import numpy as np
import pandas as pd
from collections import Counter
import nltk
import warnings
warnings.filterwarnings("ignore")
nltk.download('punkt')
stop_words = get_stopswords()
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout,GRU,LSTM,Embedding,Lambda,Flatten
from keras.utils import to_categorical 
import pandas as pd 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Using TensorFlow backend.


# preprocess a file 

In [0]:
df = pd.read_csv("data.csv")
train_data = np.array(df.text.copy())
def isNaN(num):
    return num != num
len(train_data)
for i  in range(len(train_data)): 
    if isNaN(train_data[i]):
        train_data[i]=""
vec = TfidfVectorizer(decode_error='ignore', strip_accents='unicode', encoding='utf-8',ngram_range=(1,2), min_df=5, max_df=100,max_features=512)
X = vec.fit_transform(train_data)
top_words = np.max(list(vec.vocabulary_.values()))
from sklearn.model_selection import train_test_split
X_train,X_test,y_train2,y_test2  = train_test_split(X,df.label,test_size=0.33)

#clf = LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=1000)
#clf.fit(X_train, y_train)

print(np.shape(X_test.toarray())) 

def transform_one_hot(y):
    m,n = y.shape[0], 8
    Y_one_hot_train = np.zeros((m,n))
    for i in range (m):
        Y_one_hot_train[i,y[i]]=1
    return Y_one_hot_train



(702, 512)


In [0]:
y_train,y_test = transform_one_hot(np.array(y_train2)),transform_one_hot(np.array(y_test2))

In [0]:
input_max = np.shape(X_test.toarray())[1]
input_max

512

In [0]:
model = Sequential()
model.build((None,input_max))
#model.add(Embedding(top_words,64,input_length=512))
#model.add(LSTM(units=100 ))
model.add(Dense(1024,activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64,activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(8,activation='softmax'))
model.summary()

Model: "sequential_26"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_60 (Dense)             (None, 1024)              525312    
_________________________________________________________________
dropout_20 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_61 (Dense)             (None, 64)                65600     
_________________________________________________________________
dropout_21 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_62 (Dense)             (None, 8)                 520       
Total params: 591,432
Trainable params: 591,432
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train.toarray(), y_train, epochs=30, batch_size=64, validation_data=(X_test.toarray(), y_test))

Train on 1423 samples, validate on 702 samples
Epoch 1/30
1423/1423 [==============================] - 1s 760us/step - loss: 1.9219 - acc: 0.2980 - val_loss: 1.7435 - val_acc: 0.3447
Epoch 2/30
1423/1423 [==============================] - 0s 86us/step - loss: 1.6683 - acc: 0.3823 - val_loss: 1.6363 - val_acc: 0.4202
Epoch 3/30
1423/1423 [==============================] - 0s 76us/step - loss: 1.5036 - acc: 0.4498 - val_loss: 1.5453 - val_acc: 0.4501
Epoch 4/30
1423/1423 [==============================] - 0s 82us/step - loss: 1.3538 - acc: 0.5137 - val_loss: 1.4933 - val_acc: 0.4687
Epoch 5/30
1423/1423 [==============================] - 0s 88us/step - loss: 1.2311 - acc: 0.5699 - val_loss: 1.4813 - val_acc: 0.4630
Epoch 6/30
1423/1423 [==============================] - 0s 76us/step - loss: 1.1220 - acc: 0.6128 - val_loss: 1.4659 - val_acc: 0.4715
Epoch 7/30
1423/1423 [==============================] - 0s 80us/step - loss: 1.0150 - acc: 0.6634 - val_loss: 1.4988 - val_acc: 0.4786
Epoch 8

In [0]:
clf = LogisticRegression(solver='lbfgs', multi_class='auto')
clf.fit(X_train, y_train2)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
from sklearn.ensemble import  RandomForestClassifier

In [0]:
#clf = RandomForestClassifier()
#clf.fit(X_train, y_train2)


In [0]:
import math
def metric (y_pred,y_true):
  n,c = np.shape(y_true)
  
  
  s = 0
  for i in range(n) :
    for j in range(c) : 
      s -= y_true[i][j]*math.log(y_pred[i][j])
      
  return s/n

In [0]:
y_pred = clf.predict_proba(X_test)
#y_pred = model.predict(X_test.toarray())
y_pred

array([[0.04745694, 0.12936409, 0.26326474, ..., 0.03768591, 0.02274463,
        0.13887805],
       [0.02227641, 0.16137488, 0.13232209, ..., 0.02890644, 0.01673605,
        0.48176732],
       [0.01632051, 0.59498023, 0.04556856, ..., 0.02089512, 0.0149365 ,
        0.2386607 ],
       ...,
       [0.08435292, 0.28419741, 0.0794567 , ..., 0.05149004, 0.0393702 ,
        0.22099904],
       [0.0359267 , 0.37966671, 0.14127701, ..., 0.02439545, 0.02604519,
        0.22189105],
       [0.04163411, 0.12710657, 0.08506167, ..., 0.0309666 , 0.01622155,
        0.47649559]])

In [0]:
metric(y_pred,y_test)

1.4549042383298962

In [0]:
from sklearn.naive_bayes import MultinomialNB

In [0]:
 clf = MultinomialNB()
 clf.fit(X_train,y_train2)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [0]:
y_pred = clf.predict_proba(X_test)
y_pred

array([[0.13081345, 0.2252012 , 0.0422756 , ..., 0.04467599, 0.01467682,
        0.29255077],
       [0.0724493 , 0.11881653, 0.21234795, ..., 0.03568507, 0.01761929,
        0.16515761],
       [0.05171474, 0.27675263, 0.13980394, ..., 0.04649232, 0.02069051,
        0.18767728],
       ...,
       [0.00206998, 0.37762838, 0.07646266, ..., 0.01281887, 0.02142303,
        0.17978834],
       [0.07277725, 0.39447354, 0.03655508, ..., 0.04529049, 0.01991727,
        0.29928484],
       [0.02821921, 0.33452287, 0.09974231, ..., 0.05690787, 0.03826631,
        0.25278496]])

In [0]:
clf.score(X_test,y_test2)

0.5028490028490028

In [0]:
with open("test.csv", 'r') as f:
    test_hosts = f.read().splitlines()

# Load the textual content of a set of webpages for each host into the dictionary "text". 
# The encoding parameter is required since the majority of our text is french.
text = dict()
filenames = os.listdir('text/text')
for filename in filenames:
    with codecs.open(path.join('text/text/', filename), encoding='latin-1') as f: 
        text[filename] = f.read().replace("\n", "").lower()

KeyboardInterrupt: ignored

In [0]:
test_data = list()
for host in test_hosts:
    if host in text:
        test_data.append(text[host])
    else:
        test_data.append('')
        